In [0]:
# no Databricks notebook cell
import requests
import json
import time
from datetime import datetime, timezone
import math

# Spark imports
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, LongType, StructType, StructField, StringType, TimestampType
from pyspark.sql import Window


In [0]:
import requests
import json

# Base URL da API
BASE_URL = "https://api.olhovivo.sptrans.com.br/v2.1"

# 🔑 Substitua aqui pelo seu token (gerado no portal SPTrans)
TOKEN = "725dfd912a4526086c542731327cfd72341ad64352761c16aad6ff954fa4bbb1"

# Cria uma sessão (mantém cookies e autenticação)
session = requests.Session()

# 1️⃣ Autentica na API
login_url = f"{BASE_URL}/Login/Autenticar?token={TOKEN}"
login_resp = session.post(login_url)

if login_resp.text.lower().strip() == "true":
    print("✅ Autenticado com sucesso!")
else:
    raise Exception("❌ Falha na autenticação. Verifique o token.")

# 2️⃣ Chama o endpoint de posições
posicao_url = f"{BASE_URL}/Posicao"
posicao_resp = session.get(posicao_url)

# 3️⃣ Verifica resposta
posicao_resp.raise_for_status()
data = posicao_resp.json()

In [0]:
from pyspark.sql import Row

# Captura lista de linhas
linhas = data.get("l", [])

registros = []

for linha in linhas:
    linha_codigo = linha.get("c")
    linha_id = linha.get("cl")
    sentido = linha.get("sl")
    destino_ida = linha.get("lt0")
    destino_volta = linha.get("lt1")
    qtd_veiculos = linha.get("qv", 0)
    hora_consulta = data.get("hr")

    # Dentro de cada linha, há uma lista de veículos
    for v in linha.get("vs", []):
        registros.append({
            "hora_consulta": hora_consulta,
            "linha_cod": linha_codigo,
            "linha_id": linha_id,
            "sentido": sentido,
            "lt0": destino_ida,
            "lt1": destino_volta,
            "qv": qtd_veiculos,
            "prefixo": v.get("p"),
            "acessivel": v.get("a"),
            "timestamp_api": v.get("ta"),
            "lat": v.get("py"),
            "lon": v.get("px"),
            "sv": v.get("sv"),
            "is": v.get("is")
        })

from pyspark.sql.types import *

# Define schema explícito
schema = StructType([
    StructField("hora_consulta", StringType(), True),
    StructField("linha_cod", StringType(), True),
    StructField("linha_id", IntegerType(), True),
    StructField("sentido", IntegerType(), True),
    StructField("lt0", StringType(), True),
    StructField("lt1", StringType(), True),
    StructField("qv", IntegerType(), True),
    StructField("prefixo", StringType(), True),
    StructField("acessivel", BooleanType(), True),
    StructField("timestamp_api", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True),
    StructField("sv", StringType(), True),
    StructField("is", StringType(), True)
])

# Cria DataFrame com schema fixo
df_pos = spark.createDataFrame(registros, schema=schema)


In [0]:
df_pos.write.mode("append").saveAsTable("sp_trans2.bronze.api_posicao")